This notebook is to go through the MIP API for a classification project to determine how we might use it.

We want to provide sensible defaults as well. We'll make some assumptions around cut points etc.

In [2]:
%load_ext lab_black

In [3]:
from mip import Model, INTEGER, xsum, BINARY, minimize
from sklearn.base import ClassifierMixin
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import SGDClassifier
import math
import numpy as np

In [52]:
class SLIMBinaryClassifier(ClassifierMixin):
    def __init__(self, groups=None, Lambda=100, eps=0.1):
        """
        SLIMClassifier assumes transformations are already
        applied, and we're doing integer programming

        Prior transformations are applied in the
        SLIMTransformer.

        TODO: Group sparsity constraints?
        """
        self.m = Model()
        self.Lambda = Lambda
        self.eps = eps

        self.groups = groups
        self.lambda_ = None
        self.M = None  # Lambda * max (X)

    def _add_bias(self, X):
        # adds bias to feature matrix
        return np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

    def _update_label(self, y):
        # makes it -1, 1 rather than 0, 1
        return (y - 0.5) * 2

    def fit(self, X, y):
        X = self._add_bias(X)
        y = self._update_label(y)
        n_instances = X.shape[0]
        n_feats = X.shape[1]

        # variables N + 3P
        self.lambda_ = self.m.add_var_tensor(
            shape=(n_feats,), name="lambda", var_type=INTEGER
        )
        self.alpha_ = self.m.add_var_tensor(
            shape=(n_instances,), name="alpha", var_type=BINARY
        )
        self.beta_ = self.m.add_var_tensor(
            shape=(n_feats,), name="beta", var_type=BINARY
        )
        self.gamma_ = self.m.add_var_tensor(shape=(n_feats,), name="gammas")

        # constants
        self.M = self.Lambda * np.max(X)
        self.C_0 = 0.01
        self.C_1 = 0.01

        # add constraints.
        self.m += self.lambda_ <= self.Lambda * self.beta_
        self.m += self.lambda_ >= -self.Lambda * self.beta_
        self.m += self.lambda_ <= self.gamma_
        self.m += self.lambda_ >= -self.gamma_

        #         for i in range(n_feats):
        #             self.m += self.lambda_[i] <= self.Lambda * self.beta_[i]
        #             self.m += self.lambda_[i] >= -self.Lambda * self.beta_[i]

        #             self.m += self.lambda_[i] <= self.gamma_[i]
        #             self.m += self.lambda_[i] >= -self.gamma_[i]

        #         for i in range(n_instances):
        #             self.m += (
        #                 y[i] * (self.lambda_.dot(X[i, :]))
        #                 <= self.M * (1 - self.alpha_[i]) + self.eps
        #             )
        #             self.m += (
        #                 y[i] * (self.lambda_.dot(X[i, :]))
        #                 >= -self.M * (self.alpha_[i]) + self.eps
        #             )

        self.m += (
            y * X.dot(self.lambda_.reshape(-1, 1))
            <= self.M * (1 - self.alpha_) + self.eps
        )
        self.m += (
            y * X.dot(self.lambda_.reshape(-1, 1)) >= -self.M * (self.alpha_) + self.eps
        )

        # self.m.objective = (
        #    self.alpha_.mean()
        #    + self.C_0 * self.beta_.sum()
        #    + self.C_1 * self.gamma_.sum()
        # )
        self.m.objective = self.alpha_.mean()
        self.m.optimize()
        return self

    def raw_predict(self, X):
        # X = self._add_bias(X)
        n_instances = X.shape[0]
        n_feats = X.shape[1]

        return [
            sum([X[i, ix] * self.lambda_[ix].x for ix in range(n_feats)])
            for i in range(n_instances)
        ]

In [53]:
svm_mod = SGDClassifier()

In [54]:
data = load_breast_cancer()

In [55]:
svm_mod.fit(data.data, data.target)
svm_mod.score(data.data, data.target)

0.929701230228471

In [56]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [57]:
X, _, y, _ = train_test_split(
    data.data, data.target, stratify=data.target, random_state=1, train_size=0.2
)

In [58]:
mod = SLIMBinaryClassifier(eps=0.1)
mod.fit(X, y)

In [59]:
np.vectorize(lambda var: var.x)(mod.lambda_)

LinExprTensor([  0.,   0.,   0.,   0.,   0.,   0.,   0., 100.,   0.,   0.,
                 0.,   0.,   0.,   0.,   0.,  14.,   0.,   0.,   0.,   0.,
                 0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
                 0.])

In [60]:
# get best cut point
raw_score = np.array(mod.raw_predict(data.data))
min_s, max_s = min(raw_score), max(raw_score)
for cutpoint in range(int(min_s), int(max_s) + 1):
    pred_target = raw_score > cutpoint
    print(cutpoint, accuracy_score(data.target, pred_target * 1))

0 0.6274165202108963
1 0.5536028119507909
2 0.4288224956063269
3 0.2565905096660808
4 0.14938488576449913
5 0.0984182776801406
6 0.09666080843585237
7 0.12302284710017575
8 0.14235500878734622
9 0.18804920913884007
10 0.2460456942003515
11 0.27943760984182775
12 0.30404217926186294
13 0.31985940246045697
14 0.3339191564147627
15 0.3409490333919156
16 0.3532513181019332
17 0.36203866432337434
18 0.36379613356766255
19 0.36379613356766255
20 0.37082601054481545
21 0.37082601054481545
